<a href="https://colab.research.google.com/github/nw93929/DS2002FinalProject/blob/main/etl_pipeline_setup_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Need to install necessary libraries
!pip install pandas openpyxl sqlalchemy pymysql google-cloud-storage

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.2 MB/s eta 0:00:00


In [ ]:
# Loading necessary data into data frames
import pandas as pd
# import numpy as np
outcomes = pd.read_excel('WinningPartyByCounties.xlsx')
demographics2012 = pd.read_excel('demographicsswingstates2012.xlsx')
demographics2016 = pd.read_excel('demographicsswingstates2016.xlsx')
demographics2020 = pd.read_excel('demographicswingstates2020.xlsx')

In [ ]:
# Clean outcomes data
outcomes['state'] = outcomes['state'].str.capitalize()
outcomes['county_name'] = outcomes['county_name'].str.capitalize()
columns_to_drop = ['office', 'version', 'county_fips', 'mode']
outcomes_cleaned = outcomes.drop(columns=columns_to_drop)

# Validate the changes to outcomes
print(outcomes_cleaned.head())

   year    state state_po county_name     candidate       party  \
0  2012  Arizona       AZ      Apache  BARACK OBAMA    DEMOCRAT   
1  2012  Arizona       AZ     Cochise   MITT ROMNEY  REPUBLICAN   
2  2012  Arizona       AZ    Coconino  BARACK OBAMA    DEMOCRAT   
3  2012  Arizona       AZ        Gila   MITT ROMNEY  REPUBLICAN   
4  2012  Arizona       AZ      Graham   MITT ROMNEY  REPUBLICAN   

   candidatevotes  totalvotes  
0           17147       25848  
1           29497       49003  
2           29257       51731  
3           13455       21528  
4            8076       11855  


In [ ]:
# keep only necessary columns for analysis
columns_to_keep = ['County Name', 'State Name', 'Males 18-24 %', 'Females 65+ %',
                   'White alone %', 'Black or African American alone %',
                   'less than $39,999 %', 'more than $200,000 %',
                   'Regular high school diploma %', "Master's degree %",
                   'In labor force %', 'Not in labor force %']

columns_to_keep_2016 = ['County Name', 'State Name', 'Males 18-24 %', 'Females 65+ %',
                   'White alone %', 'Black or African American alone %',
                   'less than $39,999 %', 'more than $200,000 %',
                   'Regular high school diploma %', "Master's degree %",
                   'In labor force Percentage %', 'Not in labor force Percentage %']

# Clean demographics data
demographics2012_cleaned = demographics2012[columns_to_keep]
demographics2016_cleaned = demographics2016[columns_to_keep_2016].rename(
    columns={
        "In labor force Percentage %": "In labor force %",
        "Not in labor force Percentage %": "Not in labor force %"
    }
)
demographics2020_cleaned = demographics2020[columns_to_keep]

# Display cleaned demographics data
print(demographics2012_cleaned.head())
print(demographics2016_cleaned.head())
print(demographics2020_cleaned.head())

       County Name State Name  Males 18-24 %  Females 65+ %  White alone %  \
0    Apache County    Arizona       0.108152       0.127219       0.239842   
1   Cochise County    Arizona       0.106513       0.185125       0.802842   
2  Coconino County    Arizona       0.174577       0.097482       0.626210   
3      Gila County    Arizona       0.078577       0.243020       0.791377   
4    Graham County    Arizona       0.123646       0.135922       0.753684   

   Black or African American alone %  less than $39,999 %  \
0                           0.002890             0.598109   
1                           0.039384             0.438521   
2                           0.013238             0.419966   
3                           0.004454             0.523142   
4                           0.020908             0.456232   

   more than $200,000 %  Regular high school diploma %  Master's degree %  \
0              0.010349                       0.276767           0.032809   
1         

In [ ]:
# Add the 'year' column using .loc
demographics2012_cleaned = demographics2012_cleaned.copy()
demographics2016_cleaned = demographics2016_cleaned.copy()
demographics2020_cleaned = demographics2020_cleaned.copy()

demographics2012_cleaned.loc[:, 'year'] = 2012
demographics2016_cleaned.loc[:, 'year'] = 2016
demographics2020_cleaned.loc[:, 'year'] = 2020

# Clean 'county_name' and 'state' columns for compatibility with outcomes
for df in [demographics2012_cleaned, demographics2016_cleaned, demographics2020_cleaned]:
    # Remove "County" suffix and ensure consistent capitalization
    df.loc[:, 'county_name'] = df['County Name'].str.replace(r'\s*County\s*$', '', regex=True).str.strip().str.title()
    df.loc[:, 'state'] = df['State Name'].str.strip().str.title()

# # Display cleaned demographics data to verify
# print(demographics2012_cleaned.head())
# print(demographics2016_cleaned.head())
# print(demographics2020_cleaned.head())


# # Check unique values for county_name and state in outcomes
# print("Unique county names in outcomes:", outcomes_cleaned['county_name'].unique())
# print("Unique states in outcomes:", outcomes_cleaned['state'].unique())

# # Check unique values for county_name and state in demographics
# print("Unique county names in demographics2012_cleaned:", demographics2012_cleaned['county_name'].unique())
# print("Unique states in demographics2012_cleaned:", demographics2012_cleaned['state'].unique())

In [ ]:

# Merge datasets
merged_2012 = pd.merge(
    outcomes_cleaned[outcomes_cleaned['year'] == 2012],
    demographics2012_cleaned,
    on=['county_name', 'state'],
    how='inner'
)

merged_2016 = pd.merge(
    outcomes_cleaned[outcomes_cleaned['year'] == 2016],
    demographics2016_cleaned,
    on=['county_name', 'state'],
    how='inner'
)

merged_2020 = pd.merge(
    outcomes_cleaned[outcomes_cleaned['year'] == 2020],
    demographics2020_cleaned,
    on=['county_name', 'state'],
    how='inner'
)

# Save merged data
merged_2012.to_csv('merged_data_2012.csv', index=False)
merged_2016.to_csv('merged_data_2016.csv', index=False)
merged_2020.to_csv('merged_data_2020.csv', index=False)

# Verify the merged data
print("Merged 2012 Data:")
print(merged_2012.head())

print("\nMerged 2016 Data:")
print(merged_2016.head())

print("\nMerged 2020 Data:")
print(merged_2020.head())

Merged 2012 Data:
   year_x    state state_po county_name     candidate       party  \
0    2012  Arizona       AZ      Apache  BARACK OBAMA    DEMOCRAT   
1    2012  Arizona       AZ     Cochise   MITT ROMNEY  REPUBLICAN   
2    2012  Arizona       AZ    Coconino  BARACK OBAMA    DEMOCRAT   
3    2012  Arizona       AZ        Gila   MITT ROMNEY  REPUBLICAN   
4    2012  Arizona       AZ      Graham   MITT ROMNEY  REPUBLICAN   

   candidatevotes  totalvotes      County Name State Name  ...  Females 65+ %  \
0           17147       25848    Apache County    Arizona  ...       0.127219   
1           29497       49003   Cochise County    Arizona  ...       0.185125   
2           29257       51731  Coconino County    Arizona  ...       0.097482   
3           13455       21528      Gila County    Arizona  ...       0.243020   
4            8076       11855    Graham County    Arizona  ...       0.135922   

   White alone %  Black or African American alone %  less than $39,999 %  \
0   

In [ ]:
merged_data = pd.concat([merged_2012, merged_2016, merged_2020], axis=0)

# Save the merged data locally
merged_data.to_csv('merged_data.csv', index=False)
print("Merged data saved as merged_data.csv")
print(merged_data.head())

Merged data saved as merged_data.csv
   year_x    state state_po county_name     candidate       party  \
0    2012  Arizona       AZ      Apache  BARACK OBAMA    DEMOCRAT   
1    2012  Arizona       AZ     Cochise   MITT ROMNEY  REPUBLICAN   
2    2012  Arizona       AZ    Coconino  BARACK OBAMA    DEMOCRAT   
3    2012  Arizona       AZ        Gila   MITT ROMNEY  REPUBLICAN   
4    2012  Arizona       AZ      Graham   MITT ROMNEY  REPUBLICAN   

   candidatevotes  totalvotes      County Name State Name  ...  Females 65+ %  \
0           17147       25848    Apache County    Arizona  ...       0.127219   
1           29497       49003   Cochise County    Arizona  ...       0.185125   
2           29257       51731  Coconino County    Arizona  ...       0.097482   
3           13455       21528      Gila County    Arizona  ...       0.243020   
4            8076       11855    Graham County    Arizona  ...       0.135922   

   White alone %  Black or African American alone %  less tha

In [ ]:
!curl ifconfig.me

34.82.116.136

In [ ]:
!pip install google-cloud-sdk
from google.colab import auth
auth.authenticate_user()

ERROR: Could not find a version that satisfies the requirement google-cloud-sdk (from versions: none)
ERROR: No matching distribution found for google-cloud-sdk


In [ ]:
from sqlalchemy import create_engine

# Database connection details
db_username = 'root'
db_password = 'election-data-project'
public_ip = '35.199.15.59'  # Or your server IP
db_name = 'outcomes_demographics'

# Create a database engine
engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{public_ip}/{db_name}')

# # Upload the DataFrame to the database
# merged_data.to_sql('election_demographics', engine, if_exists='replace', index=False)


# print("Data successfully uploaded to the database!")

try:
  with engine.connect() as conn:
    print(f"Connected to database {db_name} at {public_ip}")
except Exception as e:
  print(f"Error connecting to database: {e}")


Error connecting to database: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '35.199.15.59' (timed out)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [ ]:
from google.cloud import storage

def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

# Example: Upload the merged data to GCS
upload_to_gcs('your-bucket-name', 'merged_data.csv', 'cleaned_data/merged_data.csv')